# Imports

## Preparatory Imports

In [ ]:
import sys
import os

src_path = "../src"
sys.path.insert(0, src_path)

hf_token = os.environ["HUGGINGFACE_TOKEN"]

In [ ]:
from datetime import datetime
import time

## Main Imports

In [ ]:
from data import load_data
from client import Client
from tester import Tester
import pickle

In [ ]:
with open("../data/runs/run2.pickle", "rb") as conversation_file:
    conversations = pickle.load(conversation_file)

# Test the Prototype

In [ ]:
data = [load_data(f"../data/processed/train{i}.json") for i in range(1, 4)]
entries = data[0] | data[1]
c = Client(
    "mistralai/Mistral-Nemo-Instruct-2407",
    token=hf_token,
)
t = Tester(c, entries)
t.conversations = conversations
indices = sorted(list(entries.keys()))

In [ ]:
N = len(t.conversations)
print(N)

In [ ]:
indices[N]

In [ ]:
print(t.conversations[indices[N-1]])
print(t.conversations[indices[N]])

In [ ]:
i = 0

In [ ]:
bad = []
while True:
    if N == len(indices):
        break
    try:
        t.run(indices[N:])
        N = len(t.conversations)
    except Exception as e:
        print()
        N = len(t.conversations)-1
        if len(entries[indices[N]].answers) != len(t.conversations[indices[N]].answers):
            print(f"removing {indices[N]}")
            del t.conversations[indices[N]]
        with open(f"../data/runs/run2_{i}.pickle", "wb") as out:
            pickle.dump(t.conversations, out)
        i += 1
        if i == 100:
            break
        if "429 Client Error" in str(e):
            while datetime.now().minute != 1:
                time.sleep(60)
        else:
            bad.append(N)
            N += 1

In [ ]:
with open("../data/runs/run2.pickle", "wb") as out:
    pickle.dump(t.conversations, out)